In [1]:
import numpy as np
import tensorflow as tf
import xarray as xr
from matplotlib import pyplot as plt
from IPython import display
from os import makedirs
%run "topo_physics.ipynb"
%run "utils.ipynb"
layers = tf.keras.layers

准备训练数据的相关函数：

In [2]:
def block_to_full(blocks, block_size, nely, nelx):
  '''将分块重组'''
  '''(batch_size, num_of_blocks, block_size**2) into (batch_size, nely, nelx)'''
  batch_size, num_of_blocks = blocks.shape[0], blocks.shape[1]
  num_of_slice_x, num_of_slice_y = nelx//block_size, nely//block_size
  blocks = tf.reshape(blocks, (batch_size, num_of_slice_y, num_of_slice_x, block_size, block_size))
  return tf.reshape(tf.experimental.numpy.moveaxis(blocks,2,3),(batch_size, nely, nelx))

from skimage.transform import resize
def sens_normalization(sens_full, sens_red_block, block_size):
  '''使用维度缩减后的单元灵敏度（粗粒）对原尺度（细粒）灵敏度进行归一化'''
  num_of_samples = sens_full.shape[0]
  nely, nelx = sens_full.shape[1], sens_full.shape[2]
  num_of_xslice, num_of_yslice = nelx//block_size, nely//block_size
  sens_red_block = sens_red_block.reshape((num_of_samples,num_of_yslice,num_of_xslice)) # (2000,20,20)
  sens_red_expanded = resize(sens_red_block, (num_of_samples,nely,nelx), order=0, preserve_range=True)
  return sens_full/sens_red_expanded

def extract_data(ds_full, ds_block):
  '''从data_random.ipynb生成的数据中提取出神经网络输入数据'''
  nely, nelx = ds_full.nely.values.shape[0], ds_full.nelx.values.shape[0]
  block_size = ds_block.yblock.values.shape[0]
  num_of_blocks = nely//block_size * nelx//block_size
  num_of_data = ds_full.number.values.shape[0]
  dens_full = ds_full.dens_full.values # (2000,60,60)
  sens_full = ds_full.sens_full.values
  sens_red_block = ds_block.sens_red_block.values
  dens_red_block = ds_block.dens_red_block.values
  outputs = sens_normalization(sens_full, sens_red_block, block_size)
  ux_red_block = ds_block.ux_red_block.values #(1800000,2,2)
  uy_red_block = ds_block.uy_red_block.values
  disp_slice = np.stack((ux_red_block, uy_red_block), axis=-1).reshape((num_of_data,num_of_blocks,8))
  return dens_full, disp_slice, outputs


def get_dataset(ds, ds_size, batch_size, train_split=0.8, val_split=0, test_split=0.2, shuffle=True, shuffle_size=10000):
    '''数据集洗牌、分割'''
    assert (train_split + test_split + val_split) == 1
    
    if shuffle:
        # Specify seed to always have the same split distribution between runs
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size).batch(batch_size) 
    val_ds = ds.skip(train_size).take(val_size).batch(batch_size)
    test_ds = ds.skip(train_size).skip(val_size).batch(batch_size)
    
    return train_ds, val_ds, test_ds

神经网络架构部分：

In [3]:
def ConvBlock(inputs, n_filters, kernel_size, strides):
    conv = layers.Conv2D(n_filters, kernel_size, strides, activation=None, 
                         padding='same', kernel_initializer='random_normal')(inputs)
    bn = layers.BatchNormalization()(conv, training=True)
    activation = layers.ReLU()(bn)
    # activation = layers.LeakyReLU()(bn)
    # activation = PReLU(bn)
    return activation

def PReLU(_x):
  alphas = tf.compat.v1.get_variable(name=None, shape=_x.get_shape()[-1],
                       initializer=tf.constant_initializer(0.0),
                        dtype=tf.float32)
  pos = tf.nn.relu(_x)
  neg = alphas * (_x - abs(_x)) * 0.5
 
  return pos + neg

def cnn_model(num_of_layers, input_size=(None,None,1), ):
  '''input (batch_size, nely, nelx), output (batch_size, nely, nelx, n_filters)'''
  inputs = layers.Input((input_size))
  conv = ConvBlock(inputs, 64, 3, 1)
  for i in range(num_of_layers-1):
    conv = ConvBlock(conv, 64, 3, 1)
  outputs = layers.Conv2D(filters=4, kernel_size=3, strides=1, activation=None, 
                         padding='same',kernel_initializer='random_normal')(conv)
  outputs = layers.ReLU()(outputs)
  # outputs = layers.LeakyReLU()(outputs)
  # outputs = PReLU(outputs)
  model = tf.keras.Model(inputs=inputs, outputs=outputs)
  return model

def features_to_block(features, block_size=2):
  '''
  将卷积得到的特征图切块
  (batch_size, nely, nelx, fillter_size)  into 
  (batch_size*(nely//block_size)*(nelx//blocksize), block_size, block_size)) blocks
  '''
  nely, nelx, n_filters = features.shape[1], features.shape[2], features.shape[3]
  num_of_slice = nelx//block_size
  transformed = tf.experimental.numpy.moveaxis(tf.reshape(features,(-1,block_size,num_of_slice,block_size,n_filters)),1,2)
  return tf.reshape(transformed,(-1,block_size,block_size,n_filters))


def fnn_model(block_size, num_of_layers):
  feature_input = layers.Input((block_size*block_size*4)) # (batch_size, block_size, block_size, n_fil)
  disp_input = layers.Input((8))  # (batch_size, 8)
  combined = layers.concatenate([feature_input, disp_input])
  z = layers.Dense(100,activation=None)(combined)
  z = layers.ReLU()(z)
  # z = layers.LeakyReLU()(z)
  # z = PReLU(z)
  for i in range(num_of_layers-1):
    z = layers.Dense(100,activation=None)(z)
    z = layers.ReLU()(z)
    # z = layers.LeakyReLU()(z)
    # z = PReLU(z)
  z = layers.Dense(block_size**2,activation=None)(z)
  # z = layers.LeakyReLU()(z)
  z = layers.ReLU()(z)
  model = tf.keras.Model(inputs=[feature_input,disp_input], outputs=z)

  return model

In [ ]:
def custom_train(block_size, cnn_model, fnn_model, loss_function, num_epoch, optimizer, batched_train, batched_test,
                 num_of_layers_1, num_of_layers_2, save_model, casename):
  '''神经网络训练使用的主函数'''
  if save_model:
    # 创建文件夹路径
    folder_paths = [f'Model/{casename}/Models', f'Model/{casename}/Weights', f'Model/{casename}/Loss history']
    for folder_path in folder_paths:          
      makedirs(folder_path, exist_ok=True)
  best_performance=1e7
  cnn_tvars = cnn_model.trainable_variables
  fnn_tvars = fnn_model.trainable_variables
  test_losses, val_losses = [], []
  nely, nelx = 60, 60
  num_of_blocks = (nely//block_size) * (nelx//block_size)
  for epoch in range(num_epoch):
    step_test_losses, step_val_losses = [], []
    #测试数据
    for step, (input_test, output_test) in enumerate(batched_test):
      batch_size = output_test.shape[0]
      dens_input, disp_input = input_test['dens_input'], input_test['disp_input']
      feature_maps = cnn_model(dens_input)
      feature_blocks = features_to_block(feature_maps, block_size)
      feature_blocks_pipeline = tf.reshape(feature_blocks,(feature_blocks.shape[0],-1))
      disp_blocks_pipeline = tf.reshape(disp_input,(-1,8))
      fnn_outputs = fnn_model([feature_blocks_pipeline, disp_blocks_pipeline])
      fnn_outputs = tf.reshape(fnn_outputs, (batch_size, num_of_blocks,block_size**2))
      output_pred = block_to_full(fnn_outputs, block_size, nely, nelx)
      val_loss = loss_function(output_test,output_pred)
      step_val_losses.append(val_loss.numpy().item())
    #训练数据
    for step, (input_train, output_train) in enumerate(batched_train):
      with tf.GradientTape() as cnn_tape, tf.GradientTape() as fnn_tape:
        batch_size = output_train.shape[0]
        dens_input, disp_input = input_train['dens_input'], input_train['disp_input']
        cnn_tape.watch(cnn_tvars)
        fnn_tape.watch(fnn_tvars)
        feature_maps = cnn_model(dens_input)
        feature_blocks = features_to_block(feature_maps, block_size)
        feature_blocks_pipeline = tf.reshape(feature_blocks,(feature_blocks.shape[0],-1))
        disp_blocks_pipeline = tf.reshape(disp_input,(-1,8))
        fnn_outputs = fnn_model([feature_blocks_pipeline, disp_blocks_pipeline])
        fnn_outputs = tf.reshape(fnn_outputs, (batch_size, num_of_blocks,block_size**2))
        output_pred = block_to_full(fnn_outputs, block_size, nely, nelx)
        loss = loss_function(output_train,output_pred)
        cnn_grads = cnn_tape.gradient(loss, cnn_tvars)
        fnn_grads = fnn_tape.gradient(loss, fnn_tvars)
        optimizer.apply_gradients(zip(cnn_grads, cnn_tvars))
        optimizer.apply_gradients(zip(fnn_grads, fnn_tvars))
        step_test_losses.append(loss.numpy().item())

    epoch_test_loss, epoch_val_loss = tf.reduce_mean(step_test_losses), tf.reduce_mean(step_val_losses)
    if epoch_val_loss < best_performance and save_model:
      cnn_model.save_weights("Model/"+casename+"/Weights/cnn_{layers_1}+{layers_2}layers_{Nb}xblock.h5".format(layers_1=num_of_layers_1,layers_2=num_of_layers_2, Nb=block_size))
      fnn_model.save_weights("Model/"+casename+"/Weights/fnn_{layers_1}+{layers_2}layers_{Nb}xblock.h5".format(layers_1=num_of_layers_1,layers_2=num_of_layers_2, Nb=block_size))
      best_performance = epoch_val_loss
    # if epoch > 1 and epoch % 25 == 0:  
    #   cnn_model.load_weights(casename+"/Weights/cnn_{layers_1}+{layers_2}layers_{Nb}xblock.h5".format(layers_1=num_of_layers_1,layers_2=num_of_layers_2, Nb=block_size))
    #   fnn_model.load_weights(casename+"/Weights/fnn_{layers_1}+{layers_2}layers_{Nb}xblock.h5".format(layers_1=num_of_layers_1,layers_2=num_of_layers_2, Nb=block_size))
    #   cnn_model.save(casename+"/Models/cnn_{layers_1}+{layers_2}layers_{Nb}xblock_{epoch}epoch.h5".format(layers_1=num_of_layers_1,layers_2=num_of_layers_2, Nb=block_size, epoch=epoch))
    #   fnn_model.save(casename+"/Models/fnn_{layers_1}+{layers_2}layers_{Nb}xblock_{epoch}epoch.h5".format(layers_1=num_of_layers_1,layers_2=num_of_layers_2, Nb=block_size, epoch=epoch)) 
    test_losses.append(epoch_test_loss)
    val_losses.append(epoch_val_loss)
    print('epoch:',epoch,'loss:',float(epoch_test_loss),'val_loss:',float(epoch_val_loss))
  if save_model:
    cnn_model.load_weights("Model/"+casename+"/Weights/cnn_{layers_1}+{layers_2}layers_{Nb}xblock.h5".format(layers_1=num_of_layers_1,layers_2=num_of_layers_2, Nb=block_size))
    fnn_model.load_weights("Model/"+casename+"/Weights/fnn_{layers_1}+{layers_2}layers_{Nb}xblock.h5".format(layers_1=num_of_layers_1,layers_2=num_of_layers_2, Nb=block_size))
    cnn_model.save("Model/"+casename+"/Models/cnn_{layers_1}+{layers_2}layers_{Nb}xblock.h5".format(layers_1=num_of_layers_1,layers_2=num_of_layers_2, Nb=block_size))
    fnn_model.save("Model/"+casename+"/Models/fnn_{layers_1}+{layers_2}layers_{Nb}xblock.h5".format(layers_1=num_of_layers_1,layers_2=num_of_layers_2, Nb=block_size)) 
    np.save("Model/"+casename+"/Loss history/test_{layers_1}+{layers_2}layers_{Nb}xblock.npy".format(layers_1=num_of_layers_1,layers_2=num_of_layers_2, Nb=block_size),test_losses)
    np.save("Model/"+casename+"/Loss history/val_{layers_1}+{layers_2}layers_{Nb}xblock.npy".format(layers_1=num_of_layers_1,layers_2=num_of_layers_2, Nb=block_size),val_losses)
  print("best performance:", best_performance)
  return test_losses, val_losses